In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


#Initialize
team_id_pattern = r"\/verein\/(\d+)\/saison_id\/\d+"
player_id_pattern = r"\/profil\/spieler\/(\d+)"
HEADERS = {'User-Agent': 'Mozilla/5.0'}

def cleaner(string:str)->str:
    if type(string) == str:
        string = string.replace("\xa0"," ").replace('\u00e9','').replace('\u011f','').replace("\n"," ").replace("  ","").replace('\u00e1','').replace(":","").replace('\u20ac','')
        return string.strip()
    else:
        return string

def team_id_extractor(url_inp:str)->str:
    match = re.search(team_id_pattern, url_inp)
    team_id = '0'
    if match:
        team_id = match.group(1)
    return(team_id)

def player_id_extractor(url_inp:str)->str:
    match = re.search(player_id_pattern, url_inp)
    player_id = '0'
    if match:
        player_id = match.group(1)
    return(player_id)

with open('links.txt', 'r') as file:
    urls = [line.strip() for line in file.readlines()]
    
titles = ['Player_id', 'Season','Date','Left','Joined','MV','Fee','origin_id','destination_id'] 
players_transfers = pd.DataFrame(columns=titles)

class_list = ["grid__cell grid__cell--center tm-player-transfer-history-grid__season",
              'grid__cell grid__cell--center tm-player-transfer-history-grid__date',
              "grid__cell grid__cell--center tm-player-transfer-history-grid__old-club",
              "grid__cell grid__cell--center tm-player-transfer-history-grid__new-club",
              "grid__cell grid__cell--center tm-player-transfer-history-grid__market-value",
              "grid__cell grid__cell--center tm-player-transfer-history-grid__fee"]

for count,url in enumerate((urls)):
    print(f"{count+1}/{len(urls)}", end="\r")
    player_id = player_id_extractor(url)
    url = url.replace("profil", "transfers")
    html_content = requests.get(url,headers=HEADERS).text
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")
    # soup.find_all("div",class_="grid__cell grid__cell--center tm-player-transfer-history-grid__market-value")

    rows = []
    table = soup.find_all('div' , class_= "box viewport-tracking")[0]
    for i in table.find_all('div' , class_="grid tm-player-transfer-history-grid"):
        cells = []
        cells.append(player_id)
        for j in range(6):
            cells.append(cleaner(i.find('div' , class_=class_list[j]).text))
        
        try:
            cells.append(team_id_extractor(i.find_all('a' , class_='tm-player-transfer-history-grid__club-link')[0].get('href')))
        except:
            cells.append('0')
        
        try:
            cells.append(team_id_extractor(i.find_all('a' , class_='tm-player-transfer-history-grid__club-link')[1].get('href')))
        except: 
            cells.append('0')
        rows.append(cells)
    df = pd.DataFrame(rows, columns=titles)
    players_transfers = players_transfers.append(df, ignore_index=True)
players_transfers = players_transfers.applymap(cleaner)
players_transfers.to_json('players_transfers.json', orient='records', indent=4)

In [1]:
data = pd.read_json('players_transfers.json')
df = pd.DataFrame(data)

df